In [2]:
from pyspark.sql import SparkSession
#spark =  SparkSession.builder.appName('Processing').getOrCreate()
spark =  SparkSession.builder.appName('Processing')\
.master("yarn")\
.config("spark.executor.memory","12g")\
.config("spark.executor.cores","2")\
.config("spark.executor.instances","3")\
.getOrCreate()
print(spark)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 16:21:32 INFO SparkEnv: Registering MapOutputTracker
24/06/22 16:21:32 INFO SparkEnv: Registering BlockManagerMaster
24/06/22 16:21:32 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/06/22 16:21:33 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
from pyspark.sql.functions import isnull, when, count, col, desc

In [4]:
df = spark.read.csv("hdfs:///user/data/pvmdstorage/Flights_2021_1.csv",header=True)
for i in range(2,13):
    df=df.union(spark.read.csv(f"hdfs:///user/data/pvmdstorage/Flights_2021_{i}.csv",header=True))
df.count()

6311871

In [4]:
#numaram valorile nule din DepDelayMinutes
df.select(count(when(isnull("DepDelayMinutes"),"DepDelayMinutes"))).show()
df.select(count(when(isnull("ArrDelayMinutes"),"ArrDelayMinutes"))).show()

+-------------------------------------------------------------------+
|count(CASE WHEN (DepDelayMinutes IS NULL) THEN DepDelayMinutes END)|
+-------------------------------------------------------------------+
|                                                             108413|
+-------------------------------------------------------------------+

+-------------------------------------------------------------------+
|count(CASE WHEN (ArrDelayMinutes IS NULL) THEN ArrDelayMinutes END)|
+-------------------------------------------------------------------+
|                                                             126001|
+-------------------------------------------------------------------+



In [5]:
df=df.select(*(col(c).cast("float").alias(c) for c in df.columns))

In [32]:
#Determinam coloanele care au cele mai mari corelatii cu coloana ArrDelay
for column in df.columns:
    correlation=df.stat.corr(column,"ArrDelayMinutes")
    print(f"{column}: {correlation}")

Year: nan
Quarter: 0.023337430831709513
Month: 0.024645759396537747
DayofMonth: 0.0047066972077218
DayOfWeek: 0.00023115613608019217
FlightDate: nan
Marketing_Airline_Network: nan
Operated_or_Branded_Code_Share_Partners: nan
DOT_ID_Marketing_Airline: -0.02481471185520768
IATA_Code_Marketing_Airline: nan
Flight_Number_Marketing_Airline: 0.01670329707122373
Originally_Scheduled_Code_Share_Airline: nan
DOT_ID_Originally_Scheduled_Code_Share_Airline: 0.023565365250150596
IATA_Code_Originally_Scheduled_Code_Share_Airline: nan
Flight_Num_Originally_Scheduled_Code_Share_Airline: 0.02408811176475596
Operating_Airline : nan
DOT_ID_Operating_Airline: -0.01806040149379832
IATA_Code_Operating_Airline: nan
Tail_Number: nan
Flight_Number_Operating_Airline: 0.01669536140763225
OriginAirportID: -0.0022179493696779717
OriginAirportSeqID: -0.0022180443811063984
OriginCityMarketID: -0.009431711240518182
Origin: nan
OriginCityName: nan
OriginState: nan
OriginStateFips: 0.0008312922101911207
OriginStateNam

In [6]:
# Vom utiliza coloanele DepDelayMinutes,CarrierDelay,WeatherDelay,NASDelay,LateAircraftDelay

from pyspark.ml.feature import Imputer
imputer = Imputer()
imputer.setInputCols(["DepDelayMinutes","CarrierDelay","WeatherDelay","NASDelay","LateAircraftDelay","ArrDelayMinutes"])
imputer.setOutputCols(["out_DepDelayMinutes","out_CarrierDelay","out_WeatherDelay","out_NASDelay","out_LateAircraftDelay","out_ArrDelayMinutes"])
model=imputer.fit(df)
imputed_df=model.transform(df)

In [7]:
imputed_df.select(count(when(isnull("out_DepDelayMinutes"),"out_DepDelayMinutes"))).show()
imputed_df.select(count(when(isnull("out_CarrierDelay"),"out_CarrierDelay"))).show()
imputed_df.select(count(when(isnull("out_WeatherDelay"),"out_WeatherDelay"))).show()
imputed_df.select(count(when(isnull("out_NASDelay"),"out_NASDelay"))).show()
imputed_df.select(count(when(isnull("out_LateAircraftDelay"),"out_LateAircraftDelay"))).show()
imputed_df.select(count(when(isnull("out_ArrDelayMinutes"),"out_ArrDelayMinutes"))).show()

+---------------------------------------------------------------------------+
|count(CASE WHEN (out_DepDelayMinutes IS NULL) THEN out_DepDelayMinutes END)|
+---------------------------------------------------------------------------+
|                                                                          0|
+---------------------------------------------------------------------------+

+---------------------------------------------------------------------+
|count(CASE WHEN (out_CarrierDelay IS NULL) THEN out_CarrierDelay END)|
+---------------------------------------------------------------------+
|                                                                    0|
+---------------------------------------------------------------------+

+---------------------------------------------------------------------+
|count(CASE WHEN (out_WeatherDelay IS NULL) THEN out_WeatherDelay END)|
+---------------------------------------------------------------------+
|                               

In [7]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["out_DepDelayMinutes","out_CarrierDelay","out_WeatherDelay","out_NASDelay","out_LateAircraftDelay"], outputCol='features')
output = assembler.transform(imputed_df)
final_df=output.select('features','out_ArrDelayMinutes')
final_df.show(5)
train_data, test_data = final_df.randomSplit([0.7, 0.3])

+--------------------+-------------------+
|            features|out_ArrDelayMinutes|
+--------------------+-------------------+
|[3.0,26.829744338...|                0.0|
|[0.0,26.829744338...|                0.0|
|[7.0,26.829744338...|                0.0|
|[0.0,26.829744338...|                0.0|
|[0.0,26.829744338...|               14.0|
+--------------------+-------------------+
only showing top 5 rows



In [8]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql import functions as F
import time
lr = LinearRegression(labelCol='out_ArrDelayMinutes')

start=time.time()
lr_model = lr.fit(train_data)
end=time.time()
duration=end-start
print("Training time (s): ", duration)
test_results = lr_model.evaluate(test_data)
test_results.residuals.orderBy(F.col('residuals').desc()).show()

24/06/22 16:24:40 WARN Instrumentation: [466ca35d] regParam is zero, which might cause numerical instability and overfitting.


Training time (s):  45.54967260360718


+------------------+
|         residuals|
+------------------+
| 330.7836148668674|
|282.07723972209305|
|178.12983128999366|
|174.01464807541248|
| 170.4872306925384|
|169.01463292992858|
| 163.0887492886651|
| 161.5984026040411|
|161.25809391460615|
| 157.6226663808742|
|157.44870403903928|
|157.43515345741912|
|155.80056229324487|
|148.29178648091846|
|147.02332290557024|
|144.83371587784765|
|140.06954313852984|
|139.41942746288746|
|136.16884908467554|
|135.83577867253445|
+------------------+
only showing top 20 rows



In [9]:
test_results.rootMeanSquaredError

9.323882152315441

In [10]:
test_results.r2

0.9592851628385844

In [11]:
from pyspark.sql.functions import col, abs

predictions = lr_model.transform(test_data)
predictions = predictions.withColumn("absolute_percentage_error", abs((col("out_ArrDelayMinutes") - col("prediction")) / col("out_ArrDelayMinutes")))
mape = predictions.selectExpr("avg(absolute_percentage_error) as mape").collect()[0]["mape"] * 100

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
predictions.orderBy(desc("features")).show()

Mean Absolute Percentage Error (MAPE): 78.27%


+--------------------+-------------------+------------------+-------------------------+
|            features|out_ArrDelayMinutes|        prediction|absolute_percentage_error|
+--------------------+-------------------+------------------+-------------------------+
|[2554.0,2554.0,0....|             2565.0|2502.0631840931887|      0.02453677033403948|
|[2369.0,2365.0,0....|             2371.0| 2317.387545193687|       0.0226117481258174|
|[2112.0,2112.0,0....|             2135.0|2072.7400471673764|      0.02916157041340686|
|[1961.0,1877.0,0....|             1934.0|1913.4864113917768|     0.010606819342411191|
|[1915.0,20.0,0.0,...|             1890.0|1834.4033436840577|      0.02941622027298532|
|[1773.0,1773.0,0....|             1782.0| 1735.344216740992|       0.0261816965538766|
|[1757.0,22.0,1740...|             1762.0|1760.4661367088597|     8.705240017822238E-4|
|[1710.0,1710.0,0....|             1722.0|1674.6023082309414|      0.02752479196809443|
|[1707.0,224.0,0.0...|          

In [12]:
# Decision Tree

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import Imputer
import time

dt = DecisionTreeRegressor(featuresCol="features", labelCol="out_ArrDelayMinutes",maxDepth=30)

start=time.time()
model = dt.fit(train_data)
end=time.time()
duration=end-start
print("Training time (s):",duration)
predictions = model.transform(test_data)
predictions.show(50)

24/06/22 16:28:20 WARN DAGScheduler: Broadcasting large task binary with size 1135.2 KiB
24/06/22 16:28:23 WARN DAGScheduler: Broadcasting large task binary with size 1574.2 KiB
24/06/22 16:28:27 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
24/06/22 16:28:31 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
24/06/22 16:28:36 WARN DAGScheduler: Broadcasting large task binary with size 3.5 MiB
24/06/22 16:28:41 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
24/06/22 16:28:46 WARN DAGScheduler: Broadcasting large task binary with size 4.8 MiB
24/06/22 16:28:51 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
24/06/22 16:28:57 WARN DAGScheduler: Broadcasting large task binary with size 5.5 MiB
24/06/22 16:29:02 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB
24/06/22 16:29:07 WARN DAGScheduler: Broadcasting large task binary with size 5.8 MiB
24/06/22 16:29:12 WARN DAGScheduler: Broadcastin

Training time (s): 151.82955145835876


24/06/22 16:29:38 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


+--------------------+-------------------+------------------+
|            features|out_ArrDelayMinutes|        prediction|
+--------------------+-------------------+------------------+
|      (5,[0],[17.0])|               17.0|16.642857142857142|
|      (5,[0],[18.0])|               17.0| 18.19047619047619|
|      (5,[0],[33.0])|               18.0|22.897832817337463|
|      (5,[0],[48.0])|               34.0| 31.50925925925926|
|      (5,[0],[50.0])|               35.0| 31.50925925925926|
|     (5,[0],[135.0])|              119.0|151.54088050314465|
|(5,[0,1],[15.0,15...|               15.0|16.551666666666666|
|(5,[0,1],[17.0,17...|               17.0|16.592814371257486|
|(5,[0,1],[18.0,15...|               15.0|16.780262143407864|
|(5,[0,1],[18.0,16...|               16.0|16.780262143407864|
|(5,[0,1],[18.0,16...|               16.0|16.780262143407864|
|(5,[0,1],[18.0,17...|               17.0|16.780262143407864|
|(5,[0,1],[18.0,18...|               18.0|16.780262143407864|
|(5,[0,1

In [13]:
predictions.orderBy(desc("features")).show()
predictions = predictions.withColumn("absolute_percentage_error", abs((col("out_ArrDelayMinutes") - col("prediction")) / col("out_ArrDelayMinutes")))
mape = predictions.selectExpr("avg(absolute_percentage_error) as mape").collect()[0]["mape"] * 100

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

24/06/22 16:30:27 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


+--------------------+-------------------+------------------+
|            features|out_ArrDelayMinutes|        prediction|
+--------------------+-------------------+------------------+
|[2554.0,2554.0,0....|             2565.0|214.19537934968625|
|[2369.0,2365.0,0....|             2371.0|160.47692307692307|
|[2112.0,2112.0,0....|             2135.0|  219.010101010101|
|[1961.0,1877.0,0....|             1934.0|300.82123992215736|
|[1915.0,20.0,0.0,...|             1890.0|152.83197947840958|
|[1773.0,1773.0,0....|             1782.0|214.19537934968625|
|[1757.0,22.0,1740...|             1762.0|233.89655172413794|
|[1710.0,1710.0,0....|             1722.0|200.71485943775102|
|[1707.0,224.0,0.0...|             1682.0|300.82123992215736|
|[1676.0,1676.0,0....|             1678.0| 217.0924369747899|
|[1670.0,1670.0,0....|             1747.0|  285.675816993464|
|[1646.0,1646.0,0....|             1671.0|255.84848484848484|
|[1634.0,1634.0,0....|             1646.0|200.71485943775102|
|[1605.0

24/06/22 16:30:46 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


Mean Absolute Percentage Error (MAPE): 40.31%


In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol='out_ArrDelayMinutes', predictionCol='prediction')
r2 = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: 'rmse'})
print("r2:",r2)
print("rmse:",rmse)

24/06/22 16:32:13 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/06/22 16:32:42 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/06/22 16:32:44 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB
24/06/22 16:33:03 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


r2: 0.6104848428322788
rmse: 29.094148072500673
